In [11]:
import pandas as pd

In [12]:
df_statewise_rape = pd.read_csv("../datasets/crime analytics/20_Victims_of_rape.csv")
df_statewise_murder = pd.read_csv("../datasets/crime analytics/32_Murder_victim_age_sex.csv")
df_cases_crime_against_women = pd.read_csv("../datasets/crime analytics/42_Cases_under_crime_against_women.csv")
df_arrests_crime_against_women = pd.read_csv("../datasets/crime analytics/43_Arrests_under_crime_against_women.csv")
df_districtwise_caw = pd.read_csv("../datasets/crime analytics/crime/42_District_wise_crimes_committed_against_women_2001_2012.csv")
df_census_data = pd.read_csv("../datasets/crime analytics/census_data/district wise centroids.csv")

In [15]:
df_census_data['State'] = df_census_data['State'].str.upper().str.strip()
df_census_data['District'] = df_census_data['District'].str.upper().str.strip()

df_districtwise_caw['STATE/UT'] = df_districtwise_caw['STATE/UT'].str.upper().str.strip()
df_districtwise_caw['DISTRICT'] = df_districtwise_caw['DISTRICT'].str.upper().str.strip()

In [16]:
df_statewise_total_cases_crime_against_women = df_cases_crime_against_women[df_cases_crime_against_women['Group_Name']=='Total Crime Against Women']
df_statewise_total_arrests_crime_against_women = df_arrests_crime_against_women[df_arrests_crime_against_women['Group_Name']=='Total Crime Against Women']
df_statewise_total_rape_victims = df_statewise_rape[df_statewise_rape['Subgroup'] == 'Total Rape Victims']
df_statewise_murder_female_victims = df_statewise_murder[df_statewise_murder['Group_Name']=='Murder - Female Victims']
df__total_statewise_murder_female_victims = df_statewise_murder_female_victims.groupby(['Area_Name', 'Year']).agg({
    'Victims_Total': 'sum',
    'Victims_Above_50_Yrs': 'sum',
    'Victims_Upto_10_15_Yrs': 'sum',
    'Victims_Upto_10_Yrs': 'sum',
    'Victims_Upto_15_18_Yrs': 'sum',
    'Victims_Upto_18_30_Yrs': 'sum',
    'Victims_Upto_30_50_Yrs': 'sum'
}).reset_index()


In [19]:
dw_caw_latlong = pd.merge(df_census_data, 
                     df_districtwise_caw, 
                     left_on=['State', 'District'], 
                     right_on=['STATE/UT', 'DISTRICT'], 
                     how='inner')
dw_caw_latlong = dw_caw_latlong.drop(columns=['STATE/UT','DISTRICT'])

In [25]:
dw_caw_latlong.to_csv("./dw_caw_latlong.csv")

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [20]:
crime_features = dw_caw_latlong[['Rape', 'Kidnapping and Abduction', 'Dowry Deaths',
                     'Assault on women with intent to outrage her modesty',
                     'Insult to modesty of Women', 'Cruelty by Husband or his Relatives',
                     'Importation of Girls']]
# Standardize the crime features
scaler = StandardScaler()
crime_features_scaled = scaler.fit_transform(crime_features)

In [23]:
# Apply KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)  # Adjust n_clusters as needed
clusters = kmeans.fit_predict(crime_features_scaled)

# Add the cluster labels back to the original DataFrame
dw_caw_latlong['Cluster'] = clusters

In [24]:
from geopy.distance import great_circle

def find_nearest_district(lat, lon, df):
    # Compute the distance between the input location and all district locations
    distances = df.apply(lambda row: great_circle((lat, lon), (row['Latitude'], row['Longitude'])).km, axis=1)
    nearest_index = distances.idxmin()  # Get the index of the nearest district
    return df.loc[nearest_index, 'Cluster']  # Return the cluster label (safety rating)

# Example usage: find the safety rating for a given location
lat, lon = 28.6139, 77.2090  # Example coordinates (New Delhi)
lat2, lon2 = 12.9716, 77.5946 # bangalore
lat3, lon3 = 28.6692, 77.4538 # Ghaziabad
safety_rating = find_nearest_district(lat3, lon3, dw_caw_latlong)
print("Safety Rating (Cluster):", safety_rating)

Safety Rating (Cluster): 3
